In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 13.6 gigabytes of available RAM



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
ROOT = '.data'
train_data = datasets.CIFAR10(root = ROOT, 
                              train = True, 
                              download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data


In [ ]:
means = train_data.data.mean(axis = (0,1,2)) / 255
stds = train_data.data.std(axis = (0,1,2)) / 255

In [ ]:
train_transforms = transforms.Compose([
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(32, padding = 2),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = means, 
                                                std = stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean = means, 
                                                std = stds)
                       ])

In [ ]:
train_data = datasets.CIFAR10(ROOT, 
                              train = True, 
                              download = True, 
                              transform = train_transforms)

test_data = datasets.CIFAR10(ROOT, 
                             train = False, 
                             download = True, 
                             transform = test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [ ]:
BATCH_SIZE =16

train_iterator = torch.utils.data.DataLoader(train_data, BATCH_SIZE, shuffle = True)

valid_iterator = torch.utils.data.DataLoader(valid_data, BATCH_SIZE, shuffle = True)

test_iterator = torch.utils.data.DataLoader(test_data, BATCH_SIZE, shuffle = True)

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(
                in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(
                in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)

In [ ]:
class ResNet(nn.Module):
    def __init__(self, in_channels, resblock, repeat, useBottleneck=False, outputs=10):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        if useBottleneck:
            filters = [64, 256, 512, 1024, 2048]
        else:
            filters = [64, 64, 128, 256, 242]

        self.layer1 = nn.Sequential()
        self.layer1.add_module('conv2_1', resblock(filters[0], filters[1], downsample=False))
        for i in range(1, repeat[0]):
                self.layer1.add_module('conv2_%d'%(i+1,), resblock(filters[1], filters[1], downsample=False))

        self.layer2 = nn.Sequential()
        self.layer2.add_module('conv3_1', resblock(filters[1], filters[2], downsample=True))
        for i in range(1, repeat[1]):
                self.layer2.add_module('conv3_%d' % (
                    i+1,), resblock(filters[2], filters[2], downsample=False))

        self.layer3 = nn.Sequential()
        self.layer3.add_module('conv4_1', resblock(filters[2], filters[3], downsample=True))
        for i in range(1, repeat[2]):
            self.layer3.add_module('conv2_%d' % (
                i+1,), resblock(filters[3], filters[3], downsample=False))

        self.layer4 = nn.Sequential()
        self.layer4.add_module('conv5_1', resblock(filters[3], filters[4], downsample=True))
        for i in range(1, repeat[3]):
            self.layer4.add_module('conv3_%d'%(i+1,),resblock(filters[4], filters[4], downsample=False))

        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.fc = torch.nn.Linear(filters[4], outputs)

    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        input = self.gap(input)
        # torch.flatten()
        # https://stackoverflow.com/questions/60115633/pytorch-flatten-doesnt-maintain-batch-size
        input = torch.flatten(input, start_dim=1)
        input = self.fc(input)

        return input

In [ ]:
from torchsummary import summary



resnet18 = ResNet(3, ResBlock, [2, 2, 2, 2], useBottleneck=False, outputs=10)
resnet18.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(resnet18, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]           9,472
         MaxPool2d-2           [-1, 64, 14, 14]               0
       BatchNorm2d-3           [-1, 64, 14, 14]             128
              ReLU-4           [-1, 64, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]          36,928
       BatchNorm2d-6           [-1, 64, 14, 14]             128
            Conv2d-7           [-1, 64, 14, 14]          36,928
       BatchNorm2d-8           [-1, 64, 14, 14]             128
          ResBlock-9           [-1, 64, 14, 14]               0
           Conv2d-10           [-1, 64, 14, 14]          36,928
      BatchNorm2d-11           [-1, 64, 14, 14]             128
           Conv2d-12           [-1, 64, 14, 14]          36,928
      BatchNorm2d-13           [-1, 64, 14, 14]             128
         ResBlock-14           [-1, 64,

In [ ]:
optimizer = optim.SGD(resnet18.parameters(), lr=0.01, momentum=0.9)
# optimizer = optim.Adam(resnet18.parameters(), lr = 1e-3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import math
import time

EPOCHS = 100

#Setting a min variable to store model with minimum loss and then store that model
min = math.inf
for e in range(EPOCHS):
  start_time = time.time()
  train_loss, train_accuracy = train(resnet18, train_iterator, optimizer, criterion, device)
  valid_loss, valid_accuracy = evaluate(resnet18, valid_iterator, criterion, device)
  print("For Epoch %d, Training Loss is %f and Training Accuracy is %f" %(e+1, train_loss, train_accuracy*100))
  print("For Epoch %d, Validation Loss is %f and Validation Accuracy is %f" %(e+1, valid_loss, valid_accuracy*100))
  #using torch.save() to store the best model i.e. the one with minimum validation loss
  if valid_loss < min:
    # torch.save(resnet18, 'best_model')
    torch.save(resnet18.state_dict(),"/content/best_model.pt")
  end_time = time.time()
  elapsed_mins, elapsed_secs = epoch_time(start_time, end_time)
  print("Time for epoch %d is %f mins %f secs \n" % (e+1, elapsed_mins, elapsed_secs))

For Epoch 1, Training Loss is 1.624351 and Training Accuracy is 41.361536
For Epoch 1, Validation Loss is 1.276394 and Validation Accuracy is 54.912141
Time for epoch 1 is 1.000000 mins 9.000000 secs 

For Epoch 2, Training Loss is 1.190405 and Training Accuracy is 57.914149
For Epoch 2, Validation Loss is 1.036205 and Validation Accuracy is 63.338658
Time for epoch 2 is 1.000000 mins 8.000000 secs 

For Epoch 3, Training Loss is 0.975760 and Training Accuracy is 65.959385
For Epoch 3, Validation Loss is 0.841853 and Validation Accuracy is 70.567093
Time for epoch 3 is 1.000000 mins 8.000000 secs 

For Epoch 4, Training Loss is 0.846640 and Training Accuracy is 70.494134
For Epoch 4, Validation Loss is 0.768051 and Validation Accuracy is 73.462460
Time for epoch 4 is 1.000000 mins 9.000000 secs 

For Epoch 5, Training Loss is 0.755466 and Training Accuracy is 73.722449
For Epoch 5, Validation Loss is 0.734512 and Validation Accuracy is 74.261182
Time for epoch 5 is 1.000000 mins 9.0000

In [ ]:
def get_predictions(model, iterator, device):
    # model.load_state_dict(torch.load("/content/best_model")) 
    # model.optimizer=optimizer
    model.eval()

    labels = []
    probs = []
    

    for x,y in iterator:
      x = x.to(device)
      y = y.to(device)
      
      y_pred = model(x)
      labels.append(y)
      probs.append(y_pred)


    #Converting the labels and probabilities obtained to a tensor
    labels = torch.cat(labels, dim = 0)
    probs = torch.cat(probs, dim = 0)

    return labels, probs

In [ ]:
resnet18.load_state_dict(torch.load("/content/best_model.pt")) 
resnet18.optimizer=optimizer

labels, probs = get_predictions(resnet18, test_iterator, device)

print("Accuracy on Test Data is %f%%" %(calculate_accuracy(probs, labels).to('cpu').detach().numpy()*100))

Accuracy on Test Data is 88.209999%
